In [1]:
!unzip '/content/drive/My Drive/20bn_dataset.zip'

Streaming output truncated to the last 5000 lines.
  inflating: 20bn/44612/00011.jpg    
  inflating: 20bn/44612/00021.jpg    
  inflating: 20bn/44612/00007.jpg    
  inflating: 20bn/44612/00009.jpg    
  inflating: 20bn/44612/00013.jpg    
  inflating: 20bn/44612/00005.jpg    
  inflating: 20bn/44612/00001.jpg    
   creating: 20bn/86016/
  inflating: 20bn/86016/00017.jpg    
  inflating: 20bn/86016/00015.jpg    
  inflating: 20bn/86016/00023.jpg    
  inflating: 20bn/86016/00019.jpg    
  inflating: 20bn/86016/00003.jpg    
  inflating: 20bn/86016/00011.jpg    
  inflating: 20bn/86016/00021.jpg    
  inflating: 20bn/86016/00007.jpg    
  inflating: 20bn/86016/00009.jpg    
  inflating: 20bn/86016/00013.jpg    
  inflating: 20bn/86016/00005.jpg    
  inflating: 20bn/86016/00001.jpg    
   creating: 20bn/35502/
  inflating: 20bn/35502/00017.jpg    
  inflating: 20bn/35502/00015.jpg    
  inflating: 20bn/35502/00023.jpg    
  inflating: 20bn/35502/00019.jpg    
  inflating: 20bn/35502/0

In [0]:
import pandas as pd
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import ReLU,GlobalMaxPooling2D
from tensorflow.keras.layers import *
import math
import random

In [0]:
LABELS={
    "Swiping Left":                     0,
    "Swiping Right":                    1,
    "Swiping Down":                     2,
    "Swiping Up":                       3,
    "Pushing Hand Away":                4,
    "Pulling Hand In":                  5,
    "Sliding Two Fingers Left":         6,
    "Sliding Two Fingers Right":        7,
    "Sliding Two Fingers Down":         8,
    "Sliding Two Fingers Up":           9,
    "Pushing Two Fingers Away":         10,
    "Pulling Two Fingers In":           11,
    "Rolling Hand Forward":             12,
    "Rolling Hand Backward":            13,
    "Turning Hand Clockwise":           14,
    "Turning Hand Counterclockwise":    15,
    "Zooming In With Full Hand":        16,
    "Zooming Out With Full Hand":       17,
    "Zooming In With Two Fingers":      18,
    "Zooming Out With Two Fingers":     19,
    "Thumb Up":                         20,
    "Thumb Down":                       21,
    "Shaking Hand":                     22,
    "Stop Sign":                        23,
    "Drumming Fingers":                 24,
    "No gesture":                       25,
    "Doing other things":               26,

}

class readData():
    def __init__(self,
                 p_train='/content/drive/My Drive/csv/jester-v1-train.csv',
                 p_validation='/content/drive/My Drive/csv/jester-v1-validation.csv'):
        
        self.p_train      = p_train
        self.p_validation = p_validation
        self.readCSV()

    def readCSV(self):
        #Olusturulan pathlerden CSV dosyalarini okur
        print("Reading CSV Files")
        self.validationCSV = pd.read_csv(self.p_validation,header=None)
        self.trainCSV      = pd.read_csv(self.p_train,header=None)
        print("Reading CSV Files is Completed")

    def getDict(self):
        #Train ve Validation videolarının idlerini tutar
        #partition{'train': ['id-1', 'id-2', 'id-3'], 'validation': ['id-4']}
        #labels{'id-1': 0, 'id-2': 1, 'id-3': 2, 'id-4': 1}
        print("Creating Dictionaries")
        partition = {
            "train": self.trainCSV.get(0).values,
            "validation": self.validationCSV.get(0).values
        }
        labels={}
        print("Creating Train/Validation Labels")
        print("Adding Train Labels")
        for i in partition['train']:
            labels[i]=LABELS[self.trainCSV[self.trainCSV[0]==i][1].values[0]]
        print("Adding Validation Labels")
        for i in partition['validation']:
            labels[i]=LABELS[self.validationCSV[self.validationCSV[0]==i][1].values[0]]
        print("Completed")

        return partition,labels

In [0]:
class dataLoader(tf.keras.utils.Sequence):
    def __init__(self,list_IDs,labels,batchSize=16,dim=(12,150,100),nChannel=3,nClass=27,shuffle=True):
        self.dim = dim                  # Her batchte alınacak verinin boyutu Frame Sayısı * Width * Height
        self.batch_size = batchSize     # Batch boyutu
        self.labels = labels            # Classların labelları
        self.list_IDs = list_IDs        # Batch leri belirleyecek CSV train/validation csv
        self.n_channels = nChannel      # Resimdeki kanal sayısı RGB=3
        self.n_classes = nClass         # Class sayısı
        self.shuffle = shuffle          # Batchteki verileri karıştırır.
        self.on_epoch_end()

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # for dongusu icinde sonraki batch icin gesture örneklerini ve labellarını cek.
        # örnekleri ve altındaki tüm frameleri cekmek icin ekstra bir fonksiyona ihtiyac var.

        for i, ID in enumerate(list_IDs_temp):
            X[i] = self.sample_reader(ID)
            y[i] = self.labels[ID]

        return X, tf.keras.utils.to_categorical(y, num_classes=self.n_classes)

    #Her bir videonun framelerini okur ve tek örnek döner
    #validation için ayrıca bir if else koyarak validation partitionda /valid den çek
    def sample_reader(self,id):
        # batchsize,framenumber,h,w,channel
        x = np.zeros((12, 150, 100, 3), dtype='uint8');
        i = 0
        for img in os.listdir('./20bn/' + str(id)):
            image = cv2.imread("./20bn/" + str(id) + "/" + str(img))
            image = cv2.resize(image, (100,150))
            x[i] = image
            i += 1
        return x

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):

        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        X, y = self.__data_generation(list_IDs_temp)

        return X, y

In [0]:
def createModel():
    model=tf.keras.Sequential()

    model.add(TimeDistributed(Conv2D(kernel_size=(3,3),filters=32,padding='same',strides=1),input_shape=(12,150,100,3)))
    model.add(TimeDistributed(ReLU()))
    model.add(BatchNormalization())

    model.add(TimeDistributed(MaxPooling2D((2,2),strides=(1,1),data_format='channels_last')))

    model.add(TimeDistributed(Conv2D(kernel_size=(3,3),filters=64,padding='same',strides=1)))
    model.add(TimeDistributed(ReLU()))
    model.add(BatchNormalization())

    model.add(TimeDistributed(MaxPooling2D((2,2),strides=(2,2),data_format='channels_last')))

    model.add(TimeDistributed(Conv2D(kernel_size=(3,3),filters=128,padding='same',strides=1)))
    model.add(TimeDistributed(ReLU()))
    model.add(BatchNormalization())

    model.add(TimeDistributed(Conv2D(kernel_size=(3,3),filters=256,padding='same',strides=1)))
    model.add(TimeDistributed(ReLU()))
    model.add(BatchNormalization())

    model.add(TimeDistributed(MaxPooling2D((2,2),strides=(2,2),data_format='channels_last')))

    model.add(TimeDistributed(Conv2D(kernel_size=(3,3),filters=512,padding='same',strides=1)))
    model.add(TimeDistributed(ReLU()))
    model.add(BatchNormalization())

    model.add(TimeDistributed(MaxPooling2D((2,2),strides=(2,2),data_format='channels_last')))

    model.add(TimeDistributed(Conv2D(kernel_size=(3,3),filters=1024,padding='same',strides=1)))
    model.add(TimeDistributed(ReLU()))
    model.add(BatchNormalization())

    model.add(TimeDistributed(GlobalMaxPooling2D()))
    model.add(TimeDistributed(Dropout(0.4)))
    

    model.add(TimeDistributed(Dense(1024)))
    model.add(LSTM(768))
    model.add(Dense(256))
    model.add(Dense(27,activation ='softmax'))


    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.SGD(lr=0.01),
                      metrics=['accuracy'])
    model.summary()


    return model

In [6]:
data_csv=readData()
partition,labels=data_csv.getDict()

training_data=dataLoader(partition['train'],labels,15,(12,150,100),3,27,False)
validation_data=dataLoader(partition['validation'],labels,15,(12,150,100),3,27,False)

Reading CSV Files
Reading CSV Files is Completed
Creating Dictionaries
Creating Train/Validation Labels
Adding Train Labels
Adding Validation Labels
Completed


In [7]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/My Drive/Complete Models/Last Model/model_last_{epoch:02d}.h5", monitor='acc', verbose=1,
                                                save_best_only=False, mode='auto',save_freq='epoch')

csv_logger = CSVLogger("/content/drive/My Drive/Complete Models/Last Model/model_last_history_log.csv", append=True)

def step_decay(epoch):
   initial_lrate = 0.01
   drop = 0.5
   epochs_drop = 10.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)



#model=createModel()
model=tf.keras.models.load_model("/content/drive/My Drive/Complete Models/Last Model/model_last_31.h5")


model.fit(x=training_data,
                    validation_data=validation_data,
                    epochs=50,callbacks=[checkpoint,csv_logger,lrate],verbose=1,initial_epoch=31)



Epoch 32/50
 439/5400 [=>............................] - ETA: 1:31:59 - loss: 1.0094 - accuracy: 0.6261

KeyboardInterrupt: ignored

In [0]:
model=tf.keras.models.load_model("/content/drive/My Drive/Complete Models/Last Model/model_last_21.h5")
y_pred = model.evaluate_generator(validation_data,985,verbose=1)
y_pred = np.argmax(y_pred,axis=1)